In [40]:
from pathlib import Path

import pandas as pd
from collections import defaultdict

pd.set_option('display.max_colwidth', None)
pd.set_option('display.width', None)

In [41]:
dir = "/Users/miay/Library/CloudStorage/OneDrive-Personal/1 Projects/CorrelAid/2023 Q1/A4D/data/Data Prep_CLEAN DATA 21JUN/sensitive_data_removed/logs"

In [42]:
files = list(Path(dir).glob("*.log"))
len(files)

131

In [43]:
data = defaultdict(list)
for file in files:
    with open(file, "r", encoding="utf-8") as fstream:
        for line in fstream.read().splitlines():
            error = None
            warn = None
            if "ERROR" in line:
                error = line.split("\t")[-1]
                
            if "WARN" in line:
                warn = line.split("\t")[-1]
            
            if error or warn: 
                data["file"].append(file.name)
                data["error"].append(error)
                data["warning"].append(warn)

In [44]:
df = pd.DataFrame(data)

In [45]:
df

,file,error,warning
0,2022_fRYzw84pI4.log,None,Non-matching column names found: instantmeterreceiveddate.
1,2022_fRYzw84pI4.log,None,Non-matching column names found: instantmeterreceiveddate.
2,2022_fRYzw84pI4.log,None,Non-matching column names found: instantmeterreceiveddate.
3,2022_fRYzw84pI4.log,None,Non-matching column names found: instantmeterreceiveddate.
4,2022_fRYzw84pI4.log,None,Non-matching column names found: instantmeterreceiveddate.
...,...,...,...
492,2021_ctHV5ZW1cH.log,None,"Non-matching column names found: complicationscreeningselectfromlist,complicationscreeningddmmmyy."
493,2021_ctHV5ZW1cH.log,None,"Non-matching column names found: complicationscreeningselectfromlist,complicationscreeningddmmmyy."
494,2021_ctHV5ZW1cH.log,None,"Non-matching column names found: complicationscreeningselectfromlist,complicationscreeningddmmmyy."
495,2021_ctHV5ZW1cH.log,None,"Non-matching column names found: complicationscreeningselectfromlist,complicationscreeningddmmmyy."


In [46]:
df.to_csv(Path(dir) / "error_report.csv", index=False)

In [47]:
df.file.nunique()

126

## Error

In [49]:
df[~df.error.isna()]

,file,error,warning
23,2022_4ty8OlqHnl.log,Could not process 2022_4ty8OlqHnl. Error = Error: row_min < row_max is not TRUE .,None
48,2019_c58zSz33vw.log,"Could not process 2019_c58zSz33vw. Error = Error in readxl::cell_limits(c(row_min + offset, NA), c(row_max + offset, : all(NA_or_pos(lr)) is not TRUE .",None
70,2018_mI19joDYsT.log,"Could not process 2018_mI19joDYsT. Error = Error in readxl::cell_limits(c(row_min + offset, NA), c(row_max + offset, : all(NA_or_pos(lr)) is not TRUE .",None
111,2018_jzMiug0b8J.log,Could not process 2018_jzMiug0b8J. Error = Error: length(month_list) > 0 is not TRUE .,None
112,2020_ZU4xQPzdwI.log,"Could not process 2020_ZU4xQPzdwI. Error = Error in readxl::cell_limits(c(row_min + offset, NA), c(row_max + offset, : all(NA_or_pos(lr)) is not TRUE .",None
118,2019_LxdrT2ogdu.log,"Could not process 2019_LxdrT2ogdu. Error = Error in readxl::cell_limits(c(row_min + offset, NA), c(row_max + offset, : all(NA_or_pos(lr)) is not TRUE .",None
145,2017_RcvmLS8rst.log,Could not process 2017_RcvmLS8rst. Error = Error in if (num_na_rows > 0) {: argument is of length zero .,None
148,2020_gRPBXBDqGk.log,"Could not process 2020_gRPBXBDqGk. Error = Error in readxl::cell_limits(c(row_min + offset, NA), c(row_max + offset, : all(NA_or_pos(lr)) is not TRUE .",None
152,2020_Teka3ZG3bC.log,"Could not process 2020_Teka3ZG3bC. Error = Error in readxl::cell_limits(c(row_min + offset, NA), c(row_max + offset, : all(NA_or_pos(lr)) is not TRUE .",None
153,2020_EyI21cdF5Q.log,"Could not process 2020_EyI21cdF5Q. Error = Error in readxl::cell_limits(c(row_min + offset, NA), c(row_max + offset, : all(NA_or_pos(lr)) is not TRUE .",None


In [85]:
# how many files could not be processed in %?
len(df[(~df.error.isna()) & (df.error.str.contains("Could not process"))]) / len(files) * 100
                                                        

44.274809160305345

In [57]:
# missing patient id
df[(~df.error.isna()) & (df.error.str.contains("patient_id"))].file.tolist()

['2021_8AZLomJvLY.log', '2017_MrVzNiPa1N.log', '2020_4LC30JyIF2.log']

In [65]:
# month list is empty?
print("\n".join(l.split(".")[0] for l in df[(~df.error.isna()) & (df.error.str.contains("month_list"))].file.tolist()))

2018_jzMiug0b8J
2022_clCDkfEnlJ
2018_u2bhs0EKKL
2018_GF2Z1V07YD
2018_PvO8bEUkvx
2021_CgRInGxnWC
2016_978aVlnjqn


In [76]:
# cannot find patient data
for year in range(2017,2023):
    subdf = df[(~df.error.isna()) & (df.error.str.contains("readxl::cell_limits"))]
    print(subdf[subdf.file.str.startswith(str(year))].file.tolist())

['2017_mIMwYzjsVH.log']
['2018_mI19joDYsT.log', '2018_oXXwfX59K5.log', '2018_rPhf4m6nJP.log', '2018_ZrxelyRom7.log', '2018_xDmXgLt7Wz.log', '2018_AuNOJHHwqM.log']
['2019_c58zSz33vw.log', '2019_LxdrT2ogdu.log', '2019_wHpDmM1KOF.log', '2019_ONpuDbcss2.log', '2019_F37hpR20g4.log', '2019_fLcvVqqL1s.log', '2019_05QM3iWkLw.log', '2019_ry1eGF9G9H.log', '2019_q2DKOT0uhq.log', '2019_3Z66IXX9IY.log', '2019_q1C3WHYCCL.log', '2019_YfO9oosfW2.log', '2019_JLleWb7wHi.log', '2019_qe9ZJXeyym.log', '2019_guBYf9vMOY.log']
['2020_ZU4xQPzdwI.log', '2020_gRPBXBDqGk.log', '2020_Teka3ZG3bC.log', '2020_EyI21cdF5Q.log', '2020_696JbCvim4.log', '2020_Xyrz2eR1mh.log', '2020_uX5Md2dEFs.log', '2020_tBPxufmm2x.log', '2020_Ox4srVevTk.log', '2020_tN6CeX4RVv.log', '2020_yygnaJxJx2.log', '2020_dBZgwqdUUU.log', '2020_6xOA4S6WQ2.log', '2020_9ZMqREN0gV.log', '2020_hM4YXJJdp8.log', '2020_xwxrxjiIRb.log']
[]
[]


In [78]:
# check problem with num_na_rows
subdf = df[(~df.error.isna()) & (df.error.str.contains("num_na_rows"))]
subdf

,file,error,warning
145,2017_RcvmLS8rst.log,Could not process 2017_RcvmLS8rst. Error = Error in if (num_na_rows > 0) {: argument is of length zero .,None
405,2017_y6zEOxVatC.log,Could not process 2017_y6zEOxVatC. Error = Error in if (num_na_rows > 0) {: argument is of length zero .,None


In [80]:
# check start_df_msd:end_df_msd
subdf = df[(~df.error.isna()) & (df.error.str.contains("start_df_msd"))]
subdf.file.tolist()

['2021_ALoFoChH4P.log',
 '2021_P2BBVNP031.log',
 '2022_rYqoXgdiwu.log',
 '2021_AcmcLAxQxF.log']